In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('sample_data.csv')
df = df.drop('sessionUID', axis=1)
scalers = {}
for i in df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s = scaler.fit_transform(df[i].values.reshape(-1,1))
    s = np.reshape(s, len(s))
    scalers['scaler_'+ i ] = scaler
    df[i] = s
training_data = []
target_data = []
total_laps = 0
lap_groups = df.groupby('currentLapNum')
total_laps += len(lap_groups)
for l in list(lap_groups.groups):
    lap = lap_groups.get_group(l)
    target_data.append(lap.pop('lap_time_remaining'))
    training_data.append(lap)
training = [x.to_numpy() for x in training_data]
target = [y.to_numpy() for y in target_data]
print(f'Total Laps: {total_laps}')
training = training[0]
target = target[0] # for simplicity use only one lap in this demo

print(training.shape)
print(target.shape)

# split into train and test sets
trainX, testX, trainY, testY = train_test_split(training, target, test_size=0.2, random_state=42, shuffle = False)

# create overlapping windows
timesteps = training.shape[0]-1
look_back = 5
batch_size = 2
train_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(trainX, trainY, length=look_back, sampling_rate=1, stride=1, batch_size=batch_size)
test_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(testX, testY, length=look_back, sampling_rate=1, stride=1, batch_size=1)


train_X, train_y = train_generator[0]
test_X, test_y = test_generator[0]

train_samples = train_X.shape[0]*len(train_generator)
test_samples = test_X.shape[0]*len(test_generator)

print("Total Records: {}".format(len(training)))
print("Number of samples in training set (.8 * n): trainX = {}".format(trainX.shape[0]))
print("Number of samples in testing set (.2 * n): testX = {}".format(testX.shape[0]))
print("Number of total samples in training feature set: {}".format(train_samples))
print("Number of samples in testing feature set: {}".format(test_samples))

Total Laps: 2
(7891, 49)
(7891,)
Total Records: 7891
Number of samples in training set (.8 * n): trainX = 6312
Number of samples in testing set (.2 * n): testX = 1579
Number of total samples in training feature set: 6308
Number of samples in testing feature set: 1574


In [2]:
# define model
learning_rate = 0.001
units = 128
epochs = 100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()
model.add(LSTM(units, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LeakyReLU(alpha=0.5))
model.add(Dropout(0.1))
model.add(Dense(1))
adam = tf.keras.optimizers.Adam(lr=learning_rate)
callback = [EarlyStopping(monitor="loss", min_delta = 0.0001, patience = 50, mode = 'auto', restore_best_weights=True)] 
model.compile(optimizer=adam, loss='mse', metrics=['mae'])
history = model.fit(train_generator, validation_data = test_generator, callbacks=callback, verbose = 2, shuffle=False, epochs=epochs) 

Epoch 1/100


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_2956]

Function call stack:
train_function -> train_function -> train_function
